<a href="https://colab.research.google.com/github/cs145442/nlp-projects-with-tf2/blob/master/multiclass_classification_without_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd
import re
import gc
import os
# print(os.listdir("../input"))
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report


Using TensorFlow backend.


In [13]:
print(os.listdir("."))

['.config', 'kaggle.json', 'scifi-stories-text-corpus.zip', 'internet_archive_scifi_v3.txt', 'sample_data']


In [7]:
! pip install -q kaggle
from google.colab import files

files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
! kaggle datasets download -d jannesklaas/scifi-stories-text-corpus

 88% 49.0M/55.7M [00:00<00:00, 58.2MB/s]
100% 55.7M/55.7M [00:00<00:00, 103MB/s] 


In [12]:
! unzip scifi-stories-text-corpus.zip

Archive:  scifi-stories-text-corpus.zip
  inflating: internet_archive_scifi_v3.txt  


In [0]:
valid_forms = ['am','are','were','was','is','been','being','be']
blank = '----'

In [0]:
def detect(tokens):
    return [t for t in tokens if t in valid_forms]
    
def replace_blank(tokens):
    return [blank if t in valid_forms else t for t in tokens]

def create_windows(tokens, window_size=3):
    X = []
    for i, word in enumerate(tokens):
        if word == blank:
            window = tokens[i-window_size:i] + tokens[i+1:i+window_size+1]
            window = ' '.join(window)
            X.append(window)    
    return X

In [0]:
f1 = open("internet_archive_scifi_v3.txt","r")
a = f1.read()
a = re.sub('[\n]', '', a)
tokens = wordpunct_tokenize(a)
y = detect(tokens)
tokens = replace_blank(tokens)
X = create_windows(tokens)
f1.close()

In [0]:
df = pd.DataFrame()
df["Text"] = X
df["Label"] = y


In [25]:
df.head()


,Text,Label
0,Publisher Editor IF published bi -,is
1,in this magazine fiction . Any,are
2,to actual persons coincidental . #,is
3,. The title selected after much,was
4,the theory it indicative of the,is


In [30]:
one_hot = pd.get_dummies(df["Label"])
df.drop(['Label'],axis=1,inplace=True)
df = pd.concat([df,one_hot],axis=1)
df.head()

,Text,am,are,be,been,being,is,was,were
0,Publisher Editor IF published bi -,0,0,0,0,0,1,0,0
1,in this magazine fiction . Any,0,1,0,0,0,0,0,0
2,to actual persons coincidental . #,0,0,0,0,0,1,0,0
3,. The title selected after much,0,0,0,0,0,0,1,0
4,the theory it indicative of the,0,0,0,0,0,1,0,0


In [0]:
df1 = df[:200000]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1["Text"].values, df1.drop(['Text'],axis=1).values, test_size=0.2, random_state=42)

In [0]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, maxlen=50)

In [0]:
sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences, maxlen=50)

In [0]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(8, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
model.fit(X_train, y_train,
                    batch_size=1024,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 144000 samples, validate on 16000 samples
Epoch 1/10
144000/144000 [==============================] - 108s 748us/step - loss: 1.3069 - accuracy: 0.5348 - val_loss: 0.8072 - val_accuracy: 0.7037
Epoch 2/10
144000/144000 [==============================] - 106s 739us/step - loss: 0.7161 - accuracy: 0.7347 - val_loss: 0.7080 - val_accuracy: 0.7326
Epoch 3/10
144000/144000 [==============================] - 101s 705us/step - loss: 0.5980 - accuracy: 0.7761 - val_loss: 0.6905 - val_accuracy: 0.7402
Epoch 4/10
144000/144000 [==============================] - 104s 724us/step - loss: 0.5236 - accuracy: 0.8042 - val_loss: 0.6953 - val_accuracy: 0.7393
Epoch 5/10
144000/144000 [==============================] - 106s 735us/step - loss: 0.4621 - accuracy: 0.8286 - val_loss: 0.7293 - val_accuracy: 0.7308
Epoch 6/10
144000/144000 [==============================] - 107s 745us/step - loss: 0.4131 - accuracy: 0.8469 - val_loss: 0.7523 - val_accuracy: 0.7363
Epoch 7/10
144000/144000 [===========

In [38]:
score = model.evaluate(X_test, y_test,
                       batch_size=256, verbose=1)
print('Test accuracy:', score[1])

40000/40000 [==============================] - 8s 188us/step
Test accuracy: 0.7226750254631042


In [0]:
preds = model.predict(X_test)

In [40]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(preds,axis=1)))

              precision    recall  f1-score   support

           0       0.59      0.49      0.54       486
           1       0.55      0.50      0.52      2889
           2       0.94      0.94      0.94      6128
           3       0.95      0.94      0.94      3336
           4       0.44      0.28      0.34       765
           5       0.51      0.46      0.49      5911
           6       0.74      0.80      0.77     15788
           7       0.60      0.58      0.59      4697

    accuracy                           0.72     40000
   macro avg       0.66      0.62      0.64     40000
weighted avg       0.72      0.72      0.72     40000

